<a href="https://colab.research.google.com/github/ajitpal/yt-automation/blob/main/TeleToYT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install telethon
!pip install google-auth google-auth-oauthlib google-auth-httplib2
!pip install google-api-python-client

In [29]:
import logging
import telethon
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import os
from google.colab import userdata
import asyncio
from google.oauth2 import service_account

class YouTubeUploader:
    def __init__(self, telegram_channel, youtube_channel_id):
        self.telegram_channel = telegram_channel
        self.youtube_channel_id = youtube_channel_id
        self.client = telethon.TelegramClient('session_name', userdata.get('api_id'), userdata.get('api_hash'))
        self.downloaded_video_ids = set()
        self.existing_files = set()

    async def start_telegram_client(self):
        await self.client.start()
        logging.info("Telegram client started successfully")

    async def stop_telegram_client(self):
        await self.client.disconnect()
        logging.info("Telegram client disconnected")

    async def download_video(self, message):
        video_id = str(message.id)
        file_name = f"{video_id}_{self.strip_word(message.message, 'Read Nature')}.mp4"
        file_path = f"/content/videos/{file_name}"

        if video_id not in self.downloaded_video_ids and file_name not in self.existing_files:
            self.downloaded_video_ids.add(video_id)
            self.existing_files.add(file_name)
            await self.client.download_media(message, file=file_path)
            print(f"Video downloaded to: {file_path}")
            logging.info(f"Video downloaded to: {file_path}")
            return file_path
        else:
            logging.info(f"Video with ID {video_id} already downloaded.")
            print(f"Video with ID {file_path} already downloaded.")
            return None

    def strip_word(self, text, word):
        if word in text:
            text = text.replace(word, '')
        return text.strip()

    async def upload_video_to_youtube(self, file_path, title, description):
        youtube = self.youtube_authenticate()

        try:
            media = MediaFileUpload(file_path, mimetype='video/mp4', resumable=True)
            request = youtube.videos().insert(
                part="snippet,status",
                body={
                    "snippet": {
                        "title": title,
                        "description": description,
                        "categoryId": "22"  # Adjust category as needed
                    },
                    "status": {
                        "privacyStatus": "public"
                    }
                },
                media_body=media
            )

            response = None
            while response is None:
                status, response = request.next_chunk()
                if status:
                    print(f"Uploaded {int(status.progress() * 100)}%")
            print(f"Upload Complete! Video ID: {response['id']}")

        except Exception as e:
            logging.error(f"Error uploading video: {e}")
            print(f"Error uploading video: {e}")
            raise  # Re-raise the exception to stop execution

    def youtube_authenticate(self):
        """Authenticate with YouTube API using service account."""
        credentials = service_account.Credentials.from_service_account_file(
            '/content/wired-ripsaw-412506-33a4c116b964.json',  # Replace with the path to your JSON key file
            scopes=["https://www.googleapis.com/auth/youtube.force-ssl"]
        )
        return build("youtube", "v3", credentials=credentials)


async def main():
    try:
        uploader = YouTubeUploader(telegram_channel='tgrealnature', youtube_channel_id='UCLl-H8EEkapilqDZ88g071w')
        await uploader.start_telegram_client()

        # Collect existing files in the videos folder
        uploader.existing_files.update(os.listdir('/content/videos'))

        counter = 0

        async for message in uploader.client.iter_messages(uploader.telegram_channel):
            if counter >= 5:
                break

            if message.video:
                logging.info(f"Processing video message: {message.message}")
                print(f"Processing video message: {message.message}")
                try:
                    file_path = await uploader.download_video(message)
                    if file_path:
                        title = uploader.strip_word(message.message, "Read Nature")
                        description = uploader.strip_word(message.message, "Read Nature") or ''
                        # await uploader.upload_video_to_youtube(file_path, title, description)
                        # os.remove(file_path)
                        logging.info(f"Video uploaded successfully")
                        print(f"Video uploaded successfully")

                        counter += 1
                except Exception as e:
                    logging.error(f"Error downloading or uploading video: {e}")
                    print(f"Error downloading or uploading video: {e}")
                    raise  # Re-raise the exception to stop execution

        logging.info("Completed processing videos")
        print("Completed processing videos")

    except Exception as e:
        logging.critical(f"An error occurred: {e}")
        print(f"An error occurred: {e}")

    finally:
        await uploader.stop_telegram_client()

# Run the main function asynchronously
await main()


Processing video message: Turquoise Katun river, Altai, Russia🇷🇺

Real Nature🐾
Video with ID /content/videos/4016_Turquoise Katun river, Altai, Russia🇷🇺

Real Nature🐾.mp4 already downloaded.
Processing video message: Incredible Pakistan 🇵🇰

Real Nature🐾
Video with ID /content/videos/4015_Incredible Pakistan 🇵🇰

Real Nature🐾.mp4 already downloaded.
Processing video message: Rainy day in forest,UK 🇬🇧

Real Nature🐾
Video with ID /content/videos/4014_Rainy day in forest,UK 🇬🇧

Real Nature🐾.mp4 already downloaded.
Processing video message: The waterfall of Harau valley, Indonesia 🇮🇩

Real Nature🐾
Video with ID /content/videos/4013_The waterfall of Harau valley, Indonesia 🇮🇩

Real Nature🐾.mp4 already downloaded.
Processing video message: Yosemite National Park, USA🇺🇸

Real Nature🐾
Video with ID /content/videos/4012_Yosemite National Park, USA🇺🇸

Real Nature🐾.mp4 already downloaded.
Processing video message: The Majestic Canyon Fjaðrárgljúfur, Исландия 🇮🇸

Real Nature🐾
Video downloaded to: /c